# Data Cleaning Notebook Template



## Import Libraries

In [ ]:
%pip install pandas matplotlib

import pandas as pd
import matplotlib

## Load Dataset

In [ ]:
# Since the dataset has no header, we manually assign column names
COLUMN_NAMES = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

df = pd.read_csv("../data/raw_dataset.csv", names=COLUMN_NAMES, skipinitialspace=True, na_values='?')


## Initial Exploration

### Dataset Info and Summary

In [ ]:
df.info()
print(df.describe(include='all'))

### Missing Values Analysis

In [ ]:
missing_values = df.isnull().sum()

### Duplicate Analysis

In [ ]:
duplicates = df.duplicated().sum()

### Before Cleaning Snapshot

In [ ]:
before_stats = {
    'shape': df.shape,
    'total_rows': len(df),
    'sample_rows': df.head(15),
    'missing_total': missing_values.sum(),
    'duplicates': duplicates,
}

print("Before Cleaning Snapshot:")
print(f"    Shape: {before_stats['shape'][0]:,} rows x {before_stats['shape'][1]} columns")
print(f"    Sample Rows:")
display(before_stats['sample_rows'])
print(f"    Total Missing Values: {before_stats['missing_total']:,} ({(before_stats['missing_total'] / before_stats['total_rows']) * 100:.2f}%)")
print(f"    Duplicate Rows: {before_stats['duplicates']:,} ({(before_stats['duplicates'] / before_stats['total_rows']) * 100:.2f}%)")

## Cleaning

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

## Handle Missing Values

In [ ]:
cols_with_missing = df_clean.columns[df_clean.isnull().any()].tolist()

if cols_with_missing:
    print(f"Columns with missing values: {cols_with_missing}\n")
    
    # Strategy: Mode imputation for categorical columns
    for col in cols_with_missing:
        if df_clean[col].dtype == 'object':
            mode_value = df_clean[col].mode()[0]
            df_clean.fillna({col: mode_value}, inplace=True)
            print(f"✓ Filled '{col}' with mode: '{mode_value}'")
        else:
            median_value = df_clean[col].median()
            df_clean.fillna({col: median_value}, inplace=True)
            print(f"✓ Filled '{col}' with median: {median_value}")
else:
    print("No missing values to handle.")

## Standardize Formats

In [ ]:
# Strip whitespace from all string columns
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col] = df_clean[col].str.strip()
    print(f"✓ Stripped whitespace from '{col}'")

# Standardize income labels
df_clean['income'] = df_clean['income'].str.replace('.', '', regex=False)
print(f"✓ Standardized 'income' labels (removed periods)")

# Convert categorical columns to category dtype
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df_clean[col] = df_clean[col].astype('category')
print(f"✓ Converted {len(categorical_cols)} columns to category dtype")

## Outlier Detection & Treatment

In [ ]:
print("Detecting and Treating Outliers...\n")

# Identify numerical columns
numerical_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Exclude ordinal/categorical numeric columns
exclude_cols = ['education-num']  # Add other columns to exclude if needed
numerical_cols = [col for col in numerical_cols if col not in exclude_cols]

print(f"Numerical columns identified: {numerical_cols}\n")

outlier_info = {}
for col in numerical_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
    outlier_count = outliers_mask.sum()
    outlier_info[col] = outlier_count
    
    print(f"'{col}': {outlier_count:,} outliers detected")
    
    # Cap outliers using IQR method
    df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)

print(f"\n✓ Outliers capped using IQR method (1.5 × IQR)")

## Remove Duplicates

In [ ]:
temp_duplicates = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()
df_clean.reset_index(drop=True, inplace=True)

## After Stats

In [ ]:
after_stats = {
    'shape': df_clean.shape,
    'total_rows': len(df_clean),
    'sample_rows': df_clean.head(15),
    'missing_total': df_clean.isnull().sum().sum(),
    'duplicates': df_clean.duplicated().sum(),
}

## Comparison

In [ ]:
# ## 3.1 Shapes Comparison

print("SHAPES COMPARISON")
print("-" * 60)
print(f"Before: {before_stats['shape'][0]:,} rows × {before_stats['shape'][1]} columns")
print(f"After:  {after_stats['shape'][0]:,} rows × {after_stats['shape'][1]} columns")
print(f"Rows removed: {before_stats['shape'][0] - after_stats['shape'][0]:,}")

# ## 3.2 Sample Rows Comparison

print("\n=== BEFORE (First 15 rows) ===")
display(before_stats['sample_rows'])

print("\n=== AFTER (First 15 rows) ===")
display(after_stats['sample_rows'])

# ## 3.3 Data Quality Metrics

print("DATA QUALITY METRICS")
print("-" * 60)
print(f"Missing Values:")
print(f"  Before: {before_stats['missing_total']:,}")
print(f"  After:  {after_stats['missing_total']:,}")
print()
print(f"Duplicates:")
print(f"  Before: {before_stats['duplicates']:,}")
print(f"  After:  {after_stats['duplicates']:,}")
print()

## Save Cleaned Dataset

In [ ]:
df_clean.to_csv("../data/cleaned_dataset.csv", index=False)

## Summary Report


In [ ]:
print("=" * 60)
print("CLEANING PIPELINE COMPLETED SUCCESSFULLY")
print("=" * 60)
print(f"✓ Rows: {before_stats['shape'][0]:,} → {after_stats['shape'][0]:,}")
print(f"✓ Missing values: {before_stats['missing_total']:,} → {after_stats['missing_total']:,}")
print(f"✓ Duplicates removed: {before_stats['duplicates']:,} → {temp_duplicates:,} → {after_stats['duplicates']:,}")
print(f"✓ Outliers treated: {sum(outlier_info.values()):,} values capped")
print("=" * 60)

# ## Verify Cleaned Dataset

# Quick verification
print("\nFinal Dataset Info:")
df_clean.info()
print(df_clean.describe(include='all'))

print("\nCleaning process complete! The dataset is now ready for analysis and modeling.")